In [1]:
!pip uninstall -y addict
!pip install addict


In [2]:
!pip show addict


Name: addict
Version: 2.4.0
Summary: Addict is a dictionary whose items can be set using both attribute and item syntax.
Home-page: https://github.com/mewwts/addict
Author: Mats Julian Olsen
Author-email: mats@plysjbyen.net
License: UNKNOWN
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [3]:
!cp /usr/local/lib/python3.10/dist-packages/addict/addict.py /content/addict.py


cp: cannot stat '/usr/local/lib/python3.10/dist-packages/addict/addict.py': No such file or directory


In [4]:
%%writefile /content/addict.py
import collections.abc

class Dict(collections.abc.MutableMapping):
    def __init__(self, *args, **kwargs):
        self._data = dict(*args, **kwargs)

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        self._data[key] = value

    def __delitem__(self, key):
        del self._data[key]

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)


Writing /content/addict.py


In [5]:
from google.colab import files
files.download('/content/addict.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
from google.colab import files
uploaded = files.upload()


Saving addict (2).py to addict (2).py


In [7]:
!cp /content/addict.py /usr/local/lib/python3.10/dist-packages/addict/addict.py


cp: cannot create regular file '/usr/local/lib/python3.10/dist-packages/addict/addict.py': No such file or directory


In [ ]:
import os
os.kill(os.getpid(), 9)


In [2]:
!pip install torch torchvision albumentations numpy seaborn matplotlib tqdm addict pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 19.5 MB/s eta 0:00:00


In [3]:

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import numpy as np
import os
import random
from collections.abc import MutableMapping
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import logging
from addict import Dict


In [4]:
def seed_everything(seed:int=42) -> None:
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_optimizer(model:torch.nn.Module, name:str="SGD", parameters:dict={}) -> torch.optim.Optimizer:
    optimizers = {
        "SGD": torch.optim.SGD,
        "AdamW": torch.optim.AdamW,
        "Adam": torch.optim.Adam,
        "RMSprop": torch.optim.RMSprop,
    }
    instance = optimizers.get(name, "SGD")
    optimizer = instance(model.parameters(), **parameters)
    return optimizer

def get_scheduler(optimizer:torch.optim.Optimizer, name:str, parameters:dict):
    schedulers = {
        "ReduceLROnPlateau": torch.optim.lr_scheduler.ReduceLROnPlateau,
        "LambdaLR": torch.optim.lr_scheduler.LambdaLR,
        "StepLR": torch.optim.lr_scheduler.StepLR,
        "ExponentialLR": torch.optim.lr_scheduler.ExponentialLR,
        "MultiplicativeLR": torch.optim.lr_scheduler.MultiplicativeLR,
        "MultiStepLR": torch.optim.lr_scheduler.MultiStepLR,
    }
    instance = schedulers[name]
    scheduler = instance(optimizer, **parameters)
    return scheduler

def accuracy_score(predictions:torch.Tensor, targets:torch.Tensor) -> torch.Tensor:
    amount = (predictions == targets).sum()
    accuracy = amount / targets.size(0)
    return accuracy

def hide_spines(ax, spines=["top", "right", "left", "bottom"]):
    for spine in spines:
        ax.spines[spine].set_visible(False)

def get_logger(name:str=__name__, format:str="[%(asctime)s][%(levelname)s]: %(message)s") -> logging.Logger:
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter(format)
    file_handler = logging.FileHandler(name)
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.INFO)
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    logger.propagate = False
    return logger


In [5]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

class PlantDiseaseDataset(Dataset):
    def __init__(self, path, augmentations=None, image_shape=(256, 256), channels="RGB"):
        self.__images_labels = []
        self.image_shape = image_shape
        self.channels = channels
        self.augmentations = augmentations

        if os.path.exists(path):
            self.labels = [label for label in os.listdir(path) if os.path.isdir(os.path.join(path, label))]
            print(f"Labels found: {self.labels}")
            for label in self.labels:
                label_path = os.path.join(path, label)
                files = [file for file in os.listdir(label_path) if file.lower().endswith(('jpg', 'png'))]
                if not files:
                    print(f"No valid image files found in {label_path}")
                for file in files:
                    image_path = os.path.join(label_path, file)
                    self.__images_labels.append((image_path, label))
            print(f"Total images: {len(self.__images_labels)}")
        else:
            raise ValueError(f"Path {path} does not exist.")

    def __len__(self):
        return len(self.__images_labels)

    def __getitem__(self, index):
        path, label = self.__images_labels[index]
        image = self._load(path)

        if self.augmentations is not None:
            image = image.permute(1, 2, 0).numpy()
            image = self.augmentations(image=image)["image"]

        label = self.labels.index(label)

        return {"image": image, "label": label}

    def _load(self, path):
        width, height = self.image_shape
        loader = A.Compose([
            A.Resize(width=width, height=height),
            ToTensorV2(),
        ])

        image_array = np.array(Image.open(path).convert(self.channels))
        return loader(image=image_array)["image"]


In [6]:
import torch
import torch.nn as nn
import torchvision.models as models

class PlantDiseaseModel(nn.Module):
    def __init__(self, classes=2):
        super(PlantDiseaseModel, self).__init__()
        self.model = models.resnet34(pretrained=True)

        for parameter in self.model.parameters():
            parameter.requires_grad = False

        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=classes),
            nn.Softmax(dim=1)
        )

    def forward(self, image):
        output = self.model(image)
        return output


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader


train_dataset = PlantDiseaseDataset(
    path="/content/drive/My Drive/cucumber_dataset /train/train",
    image_shape=(128, 128),
    channels="RGB",
    augmentations=A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        ToTensorV2()
    ])
)

validation_dataset = PlantDiseaseDataset(
    path="/content/drive/My Drive/cucumber_dataset /Validation/Validation",
    image_shape=(128, 128),
    channels="RGB",
    augmentations=A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        ToTensorV2()
    ])
)

test_dataset = PlantDiseaseDataset(
    path="/content/drive/My Drive/cucumber_dataset /test/test",
    image_shape=(128, 128),
    channels="RGB",
    augmentations=A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        ToTensorV2()
    ])
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    num_workers=2,
    pin_memory=True,
    shuffle=True
)

validation_loader = DataLoader(
    dataset=validation_dataset,
    batch_size=64,
    num_workers=2,
    pin_memory=True,
    shuffle=False
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    num_workers=2,
    pin_memory=True,
    shuffle=False
)


Labels found: ['Anthracnose lesions', 'Leaf MInner', 'Yellow Stuning', 'Healthy', 'Downy Mildew', 'Powdred']
Total images: 946
Labels found: ['Anthracnose lesions', 'Downy Mildew', 'Healthy', 'Leaf MInner', 'Powdred', 'Yellow Stuning']
Total images: 76
Labels found: ['Anthracnose lesions', 'Downy Mildew', 'Healthy', 'Leaf MInner', 'Powdred', 'Yellow Stuning']
Total images: 127


In [13]:
import torch
from tqdm import tqdm
from addict import Dict

class Trainer:
    def __init__(self, model, criterion, optimizer, metric, scheduler=None, logger=None, device="cpu"):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.logger = logger
        self.device = torch.device(device)
        self.metric = metric
        self.history = Dict({})

    def __log(self, logs):
        for k, v in logs.items():
            if k not in self.history:
                self.history[k] = []
            self.history[k].append(v)

    def evaluate(self, loader):
        loss, score, length = 0, 0, len(loader)

        self.model.to(self.device)
        with torch.no_grad():
            loop = tqdm(loader, position=0, colour="BLACK", desc="Evaluating", leave=True)
            for batch in loop:
                if self.device.type != "cpu": torch.cuda.empty_cache()
                self.model.eval()

                images = batch["image"].float().to(self.device)
                labels = batch["label"].long().to("cpu")

                probabilities = self.model(images).float().to("cpu")
                predictions = torch.argmax(probabilities, dim=1).detach()

                batch_loss = self.criterion(probabilities, labels)
                loss += batch_loss.item()

                batch_score = self.metric(predictions, labels).item()
                score += batch_score

            loss /= length
            score /= length

        return loss, score

    def fit(self, train_loader, validation_loader=None, epochs=10):
        self.model.to(self.device)
        train_length = len(train_loader)

        for epoch in range(epochs):
            epoch_loss, epoch_score = 0, 0

            loop = tqdm(train_loader, position=0, colour="BLACK", leave=True, desc=f"Epoch [{epoch+1}/{epochs}]: ")
            for batch in loop:
                if self.device.type != "cpu": torch.cuda.empty_cache()
                self.optimizer.zero_grad()
                self.model.train()

                images = batch["image"].float().to(self.device)
                labels = batch["label"].long().to("cpu")

                probabilities = self.model(images).float().to("cpu")
                predictions = torch.argmax(probabilities, dim=1).detach()

                batch_loss = self.criterion(probabilities, labels)
                epoch_loss += batch_loss.item()

                batch_score = self.metric(predictions, labels).item()
                epoch_score += batch_score

                batch_loss.backward()
                self.optimizer.step()

            epoch_loss /= train_length
            epoch_score /= train_length

            self.__log({"train_losses": epoch_loss, "train_scores": epoch_score})
            if self.logger is not None:
                self.logger.info(f"Epoch [{epoch+1}/{epochs}]: Loss: {epoch_loss} | Metric: {epoch_score}")

            if validation_loader is not None:
                validation_loss, validation_score = self.evaluate(validation_loader)
                self.__log({"validation_losses": validation_loss, "validation_scores": validation_score})
                if self.logger is not None:
                    self.logger.info(f"Validation Epoch [{epoch+1}/{epochs}]: Loss: {validation_loss} | Metric: {validation_score}")

            if self.scheduler is not None:
                if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    self.scheduler.step(validation_loss)
                else:
                    self.scheduler.step()

                if self.logger is not None:
                    lr = self.optimizer.param_groups[0]["lr"]
                    self.logger.info(f"Epoch [{epoch+1}/{epochs}] Learning Rate: {lr}")


In [14]:
import torch.nn as nn
from sklearn.metrics import accuracy_score
train_config = {
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "epochs": 10,
    "seed": 2021,
    "image_shape": (128, 128),
    "image_channels": 3,
    "num_workers": 2,
    "batch_size": 32,
    "optimizer": {
        "type": "AdamW",
        "parameters": {
            "lr": 0.001,
            "weight_decay": 0.01,
        }
    },
    "scheduler": {
        "type": "ReduceLROnPlateau",
        "parameters": {
            "patience": 2,
            "mode": "min",
            "factor": 0.1,
        }
    }
}
model = PlantDiseaseModel(classes=len(train_dataset.labels))
criterion = nn.CrossEntropyLoss()
optimizer = get_optimizer(
    model=model,
    name=train_config["optimizer"]["type"],
    parameters=train_config["optimizer"]["parameters"]
)
scheduler = None
if "scheduler" in train_config:
    scheduler = get_scheduler(
        optimizer=optimizer,
        name=train_config["scheduler"]["type"],
        parameters=train_config["scheduler"]["parameters"]
    )
trainer_logger = get_logger("trainer")
trainer = Trainer(
    model=model,
    criterion=criterion,
    metric=accuracy_score,
    optimizer=optimizer,
    scheduler=scheduler,
    logger=trainer_logger,
    device=train_config["device"]
)
trainer.fit(train_loader, validation_loader=validation_loader, epochs=train_config["epochs"])


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 199MB/s]
Epoch [1/10]:   0%|          | 0/30 [00:20<?, ?it/s]


AttributeError: 'float' object has no attribute 'item'

In [ ]:
class PlantDiseaseModel(nn.Module):
    def __init__(self, classes=2):
        super(PlantDiseaseModel, self).__init__()
        self.model = models.resnet34(pretrained=True)

        for parameter in self.model.parameters():
            parameter.require_grad = False

        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=classes),
            nn.Softmax(dim=1)
        )

    def forward(self, image):
        output = self.model(image)
        return output


In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

class Trainer:
    def __init__(self, model, criterion, optimizer, metric, scheduler=None, logger=None, device="cpu"):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.logger = logger
        self.device = torch.device(device)
        self.best_validation_loss = 0
        self.metric = metric
        self.history = Dict({})

    def __log(self, logs):
        for k, v in logs.items():
            if k not in self.history:
                self.history[k] = []
            self.history[k].append(v)

    def evaluate(self, loader):
        loss, score, length = 0, 0, len(loader)

        self.model.to(self.device)
        with torch.no_grad():
            loop = tqdm(loader, position=0, colour="BLACK", desc=f"Evaluating: ", leave=True)
            for batch in loop:
                if self.device.type != "cpu": torch.cuda.empty_cache()
                self.model.eval()
                images = batch["image"].float().to(self.device)
                labels = batch["label"].long().to("cpu")

                probabilities = self.model(images).float().to("cpu")
                predictions = torch.argmax(probabilities, dim=1).detach()

                batch_loss = self.criterion(probabilities, labels)
                loss += batch_loss.item()

                batch_score = self.metric(predictions, labels).item()
                score += batch_score

            loss /= length
            score /= length

        return loss, score

    def fit(self, train_loader, validation_loader=None, epochs=10):
        self.model.to(self.device)
        train_length = len(train_loader)

        for epoch in range(epochs):
            epoch_loss, epoch_score = 0, 0

            loop = tqdm(train_loader, position=0, colour="BLACK", leave=True, desc=f"Epoch [{epoch+1}/{epochs}]: ")
            for batch in loop:
                if self.device.type != "cpu": torch.cuda.empty_cache()
                self.optimizer.zero_grad()
                self.model.train()
                images = batch["image"].float().to(self.device)
                labels = batch["label"].long().to("cpu")

                probabilities = self.model(images).float().to("cpu")
                predictions = torch.argmax(probabilities, dim=1).detach()

                batch_loss = self.criterion(probabilities, labels)
                epoch_loss += batch_loss.item()

                batch_score = self.metric(predictions, labels).item()
                epoch_score += batch_score

                batch_loss.backward()
                self.optimizer.step()

            epoch_loss /= train_length
            epoch_score /= train_length

            self.__log({"train_losses": epoch_loss, "train_scores": epoch_score})
            if self.logger is not None:
                self.logger.info(f"Epoch [{epoch+1}/{epochs}]: Loss: {epoch_loss} | Metric: {epoch_score}")

            if validation_loader is not None:
                validation_loss, validation_score = self.evaluate(validation_loader)
                self.__log({"validation_losses": validation_loss, "validation_scores": validation_score})
                if self.logger is not None:
                    self.logger.info(f"Validation Epoch [{epoch+1}/{epochs}]: Loss: {validation_loss} | Metric: {validation_score}")

                if self.scheduler is not None:
                    if isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                        self.scheduler.step(validation_loss)
                    else:
                        self.scheduler.step()

                    if self.logger is not None:
                        lr = self.optimizer.param_groups[0]["lr"]
                        self.logger.info(f"Epoch [{epoch+1}/{epochs}] Learning Rate: {lr}")


In [ ]:
  import torch
  import torch.nn as nn
  from sklearn.metrics import accuracy_score
  train_config = {
      "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
      "epochs": 10,
      "seed": 2021,
      "image_shape": (128, 128),
      "image_channels": 3,
      "num_workers": 2,
      "batch_size": 32,
      "optimizer": {
          "type": "AdamW",
          "parameters": {
              "lr": 0.001,
              "weight_decay": 0.01,
          }
      },
      "scheduler": {
          "type": "ReduceLROnPlateau",
          "parameters": {
              "patience": 2,
              "mode": "min",
              "factor": 0.1,
          }
      }
  }
  model = PlantDiseaseModel(classes=len(train_dataset.labels))
  criterion = nn.CrossEntropyLoss()
  optimizer = get_optimizer(
      model=model,
      name=train_config["optimizer"]["type"],
      parameters=train_config["optimizer"]["parameters"]
  )
  scheduler = None
  if "scheduler" in train_config:
      scheduler = get_scheduler(
          optimizer=optimizer,
          name=train_config["scheduler"]["type"],
          parameters=train_config["scheduler"]["parameters"]
      )
  trainer_logger = get_logger("trainer")
  trainer = Trainer(
      model=model,
      criterion=criterion,
      metric=accuracy_score,
      optimizer=optimizer,
      scheduler=scheduler,
      logger=trainer_logger,
      device=train_config["device"]
  )


In [ ]:
seed_everything(train_config["seed"])


In [ ]:
trainer.fit(train_loader, validation_loader=validation_loader, epochs=train_config["epochs"])


In [ ]:
test_loss, test_accuracy = trainer.evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


In [ ]:
torch.save(model.state_dict(), "/content/drive/My Drive/plant_disease_model.pth")


In [ ]:
import torch
from PIL import Image
import numpy as np

model.eval()
sample_image = test_dataset[0]["image"].unsqueeze(0).to(train_config["device"])
if sample_image.dtype != torch.float32:
    sample_image = sample_image.to(dtype=torch.float32)

with torch.no_grad():
    prediction = model(sample_image)
    predicted_class = torch.argmax(prediction, dim=1)
    print(f"Predicted Class: {test_dataset.labels[predicted_class.item()]}")


Predicted Class: Anthracnose lesions


In [ ]:
from google.colab import files
from PIL import Image, ImageDraw, ImageFont
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt

def preprocess_image(image_path, image_shape=(128, 128), channels="RGB"):
    loader = A.Compose([
        A.Resize(width=image_shape[0], height=image_shape[1]),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    image_array = np.array(Image.open(image_path).convert(channels))
    image_tensor = loader(image=image_array)["image"]
    return image_tensor

uploaded = files.upload()
image_path = next(iter(uploaded))
image_tensor = preprocess_image(image_path, image_shape=(128, 128), channels="RGB").unsqueeze(0).to(train_config["device"])

model.eval()
with torch.no_grad():
    prediction = model(image_tensor)
    predicted_class = torch.argmax(prediction, dim=1).item()
    disease_probability = torch.softmax(prediction, dim=1)[0, predicted_class].item() * 100

predicted_class_name = train_dataset.labels[predicted_class]
predicted_text = f"{predicted_class_name} ({disease_probability:.2f}%)"

original_image = Image.open(image_path)
draw = ImageDraw.Draw(original_image)
font = ImageFont.load_default()
text_position = (10, 10)
draw.text(text_position, predicted_text, fill="red", font=font)

plt.figure(figsize=(16, 8))

plt.subplot(1, 2, 1)
plt.imshow(Image.open(image_path))
plt.axis('off')
plt.title("Input Image")

plt.subplot(1, 2, 2)
plt.imshow(original_image)
plt.axis('off')
plt.title(f"Predicted Class: {predicted_text}")

plt.show()


In [ ]:

from google.colab import files
from PIL import Image, ImageDraw, ImageFont
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt


def preprocess_image(image_path, image_shape=(128, 128), channels="RGB"):
    transform = A.Compose([
        A.Resize(height=image_shape[0], width=image_shape[1]),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    image = Image.open(image_path).convert(channels)
    image_array = np.array(image)
    transformed = transform(image=image_array)
    image_tensor = transformed["image"].unsqueeze(0)
    return image_tensor


uploaded = files.upload()
image_path = next(iter(uploaded.keys()))


image_tensor = preprocess_image(
    image_path,
    image_shape=(128, 128),
    channels="RGB"
).to(train_config["device"])


model.eval()
with torch.no_grad():
    output = model(image_tensor)
    probabilities = torch.softmax(output, dim=1)
    predicted_class_idx = torch.argmax(probabilities, dim=1).item()
    predicted_probability = probabilities[0, predicted_class_idx].item() * 100


predicted_class_name = train_dataset.labels[predicted_class_idx]
predicted_text = f"{predicted_class_name} ({predicted_probability:.2f}%)"

original_image = Image.open(image_path).convert("RGB")
draw = ImageDraw.Draw(original_image)
font = ImageFont.truetype("arial.ttf", size=24)
text_position = (10, 10)
text_color = (255, 0, 0)
draw.text(text_position, predicted_text, fill=text_color, font=font)


plt.figure(figsize=(16, 8))


plt.subplot(1, 2, 1)
plt.imshow(Image.open(image_path))
plt.axis('off')
plt.title("Input Image")


plt.subplot(1, 2, 2)
plt.imshow(original_image)
plt.axis('off')
plt.title(f"Predicted: {predicted_text}")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
epochs = range(1, len(trainer.history["train_scores"]) + 1)
train_scores = trainer.history["train_scores"]
validation_scores = trainer.history["validation_scores"]
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_scores, 'b-', label='Training Accuracy')
plt.plot(epochs, validation_scores, 'r-', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [16]:
!pip install tensorflow numpy matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 577.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00


In [18]:
from google.colab import drive
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Mount Google Drive
drive.mount('/content/drive')

# Define dataset paths
train_dir = "/content/drive/My Drive/cucumber_dataset /train/train"
val_dir = "/content/drive/My Drive/cucumber_dataset /Validation/Validation"
test_dir = "/content/drive/My Drive/cucumber_dataset /test/test"

# Verify paths exist
assert os.path.exists(train_dir), "Train directory not found!"
assert os.path.exists(val_dir), "Validation directory not found!"
assert os.path.exists(test_dir), "Test directory not found!"

# Image augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load dataset
batch_size = 32
img_size = (128, 128)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')

# Number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

# Define CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
epochs = 10
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)

# Save model
model_path = "/content/drive/My Drive/cucumber_model.h5"
model.save(model_path)
print(f"Model saved as {model_path}")

# Load and evaluate model
loaded_model = load_model(model_path)
test_loss, test_acc = loaded_model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 946 images belonging to 6 classes.
Found 76 images belonging to 6 classes.
Found 127 images belonging to 6 classes.
Number of classes: 6


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,414 (12.61 MB)

...

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 189s 6s/step - accuracy: 0.1709 - loss: 1.9018 - val_accuracy: 0.1974 - val_loss: 1.7317
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.2979 - loss: 1.6242 - val_accuracy: 0.2500 - val_loss: 1.5018
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 997ms/step - accuracy: 0.3722 - loss: 1.4640 - val_accuracy: 0.2632 - val_loss: 1.6394
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 996ms/step - accuracy: 0.4066 - loss: 1.3619 - val_accuracy: 0.3289 - val_loss: 1.3797
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 992ms/step - accuracy: 0.4606 - loss: 1.2613 - val_accuracy: 0.2632 - val_loss: 1.5576
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4577 - loss: 1.2950 - val_accuracy: 0.2632 - val_loss: 1.5307
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 1000ms/step - accuracy: 0.4873 - loss: 1.2743 - val_accuracy: 0.3684 - val_loss: 1.4161
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4689 - loss: 1.2720 - val_accuracy: 0.28

Model saved as /content/drive/My Drive/cucumber_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.4034 - loss: 1.5248
Test Accuracy: 43.31%
